Binary Sentiment (Logistic Regression) (Task 4)

Note: User will have to upload the csv to their jupyter

In [7]:
import pandas as pd

data= pd.read_csv("cleaned_data.csv") #will have to change the file name for the single large dataset

#Testing if it read from the read_csv
#print(data.head(2))
#print("\n")


Transform rating into:
Positive if rating > 3, Negative if rating ≤ 3

In [8]:
#print("Rating Before Transformation")
#print(data["rating"].head(10))

data["rating"] = data["rating"].apply(lambda r: "Positive" if r > 3 else "Negative")

#print("Rating Before Transformation")
#print(data["rating"].head(10))

#update the csv at source
data.to_csv("cleaned_data.csv", index= False)

Part a. Train/Test Split: 80% train, 20% test, randomly shuffled.

Using "text" and "rating" from the dataset to predict whether "rating" would be "Positve" or "Negative" based on the "text"

In [ ]:
from sklearn.model_selection import train_test_split

X = data["text"]
y= data["rating"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state= 10, shuffle= True) #not too sure of to keep "random_state" but it will make the evaluation results more consistent

'''
#Testing
print("Total amount of data: ", len(X))
print("The amount of data in trained split: ", len(X_train))
print("The amount of data in test split: ", len(X_test))
print("Dataset before shuffle : ", X.head(5).tolist())
print("Dataset after shuffle : ", X_train.head(5).tolist())
'''

Total amount of data:  202855
The amount of data in trained split:  162284
The amount of data in test split:  40571
Dataset before shuffle :  ["This spray is really nice. It smells really good, goes on really fine, and does the trick. I will say it feels like you need a lot of it though to get the texture I want. I have a lot of hair, medium thickness. I am comparing to other brands with yucky chemicals so I'm gonna stick with this. Try it!", 'This product does what I need it to do, I just wish it was odorless or had a soft coconut smell. Having my head smell like an orange coffee is offputting. (granted, I did know the smell was described but I was hoping it would be light)', 'Smells good, feels great!', 'Felt synthetic', 'Love it']
Dataset after shuffle :  ['this is the third time ordering this product', 'great stuff.  i always buy bulk but its nice to find this in a pinch.  thanks amazon and spir zon', 'These are the same gel packs that were used at the Chronic pain recovery center 